In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
 
 
# 导入数据集
train = pd.read_csv('./house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('./house-prices-advanced-regression-techniques/test.csv')

In [2]:
# 将测试集和训练集连接
full = pd.concat([train,test],ignore_index=True)
 
# 查看缺失值，并将缺失值的特征由大到小排列。
tmp = full.isnull().sum()
tmp[tmp>0].sort_values(ascending=False)
# 对数值类型的填充
full.fillna(full.mean(),inplace=True)
 
# 查看剩余的缺失值
tmp = train.isnull().sum()
tmp[tmp>0].sort_values(ascending=False)# 对字符类型进行填充 
full.fillna('None',inplace=True)
 
tmp = full.isnull().sum()
tmp[tmp>0].sort_values(ascending=False)
# 去掉Id，SalePrice列
full = full.drop(['Id','SalePrice'],axis=1)
 
# 变成one-hot编码
full=pd.get_dummies(full).reset_index(drop=True)

/work/wangzhen/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler,StandardScaler
 
# 标准化处理
std = StandardScaler()
full = std.fit_transform(full)
 
# 引入LinearRegression()模型
model = LinearRegression()
 
 
# 将训练集和测试集分开，
n_train = train.shape[0]#行数
 
#划分数据集
X = np.array(full[:n_train])##相当于取出X
y = np.array(train.SalePrice)#取出训练集的标签

test_x = np.array(full[n_train:])#1461行取到最后
print(X.shape)
print(y.shape)
print(test_x.shape)
 
# 训练模型
# model.fit(X,y)

(1460, 310)
(1460,)
(1459, 310)


In [4]:
#(b)Using least squares,
from sklearn.model_selection import train_test_split
x_train,x_train_test,y_train,y_train_test = train_test_split(X,y,test_size=0.15,random_state=1)
print(x_train.shape,x_train_test.shape,y_train.shape,y_train_test.shape)
print(type(x_train))

(1241, 310) (219, 310) (1241,) (219,)
<class 'numpy.ndarray'>


In [5]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
model.fit(x_train,y_train)
# model.fit(X,y)
train_test = model.predict(x_train_test)
# 预测
saleprice = model.predict(test_x)
 
# 产生要提交的csv文件
result=pd.DataFrame({'Id':test.Id, 'SalePrice':saleprice})
result.to_csv("./house-prices-advanced-regression-techniques/submission.csv",index=False)

def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse

In [6]:
print('LR mse : {}'.format(mean_squared_error(train_test,y_train_test)))

LR mse : 1.1635285013863521e+33


In [7]:
from sklearn.model_selection import cross_val_score
print('LR mse : {}'.format(rmse_cv(model,X,y)))

LR mse : [2.45149378e+17 1.15458582e+17 4.19777317e+17 1.55799857e+17
 9.14583631e+17]


In [8]:
#(c)(d)use l1 l2 regularization
from sklearn.linear_model import Ridge
model1 = Ridge(alpha=1)
model1.fit(x_train,y_train)
y_pred = model1.predict(x_train_test)
print(mean_squared_error(y_pred,y_train_test))
from sklearn.linear_model import RidgeCV
model2 = RidgeCV(alphas=[0.001,0.01,0.1,0.5,1,3,5,7,9,10])
# model2 = RidgeCV(alphas=[2])
model2.fit(x_train,y_train)
print(model2.alpha_)
y_pred2 = model2.predict(x_train_test)
print(mean_squared_error(y_pred2,y_train_test))
saleprice = model2.predict(test_x)
result=pd.DataFrame({'Id':test.Id, 'SalePrice':saleprice})
result.to_csv("./house-prices-advanced-regression-techniques/submission.csv",index=False)

855519372.7833714
10.0
769396403.2690396
